In [1]:
#Needed libraries installation (run it once)
#%pip install -r requirements.txt

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "NA"

In [ ]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.1", request_timeout=4000.0)

In [4]:
from llama_index.core import SimpleDirectoryReader


documents = SimpleDirectoryReader("data_ua_conf").load_data()



In [5]:
from transformers import AutoModel, AutoTokenizer, XLMRobertaTokenizer

save_directory = "./local_embedding_model_ua_2"

c:\automatization\conf7nov\conf_7_nov\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # Download local embedding model, should be ran only once

# model_name = "google-bert/bert-base-multilingual-cased"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)
# # Save to local directory

# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory)

In [8]:
# Load local model 
# Load from local directory
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory)

In [9]:
#create embedding function
from llama_index.core.embeddings import BaseEmbedding
import torch

class LocalEmbedding(BaseEmbedding):
    def __init__(self, model, tokenizer):
        super().__init__()
        self._model = model  # Explicitly define with a private attribute
        self._tokenizer = tokenizer

    def get_text_embedding(self, text):
        # Tokenize with truncation to the model's maximum length
        inputs = self._tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        with torch.no_grad():
            outputs = self._model(**inputs)
        # Compute mean of the last hidden state as embedding
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        return embedding

    # Implement required methods
    def _get_query_embedding(self, query):
        return self.get_text_embedding(query)

    def _get_text_embedding(self, text):
        return self.get_text_embedding(text)

    async def _aget_query_embedding(self, query):
        return self.get_text_embedding(query)

# Initialize custom embedding with your model and tokenizer
embedding_model = LocalEmbedding(model=model, tokenizer=tokenizer)


In [10]:
from llama_index.core import VectorStoreIndex, Settings


# Configure global settings
Settings.llm = llm
Settings.embed_model = embedding_model
# Set the chunk size (number of tokens per chunk)
Settings.chunk_size = 1024
# Set the chunk overlap (number of tokens overlapping between chunks)
Settings.chunk_overlap = 128
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Generating embeddings: 100%|██████████| 622/622 [08:29<00:00,  1.22it/s]


In [11]:
# Set up the retriever from your index
top_k = 5
retriever = index.as_retriever(similarity_top_k=top_k)


In [12]:
def get_augmented_prompt(query):
    retrieved_documents = retriever.retrieve(query)
    # Augment the prompt by including the retrieved documents
    # Constructing an augmented prompt with context from retrieved documents
    augmented_prompt = f"Context:\n"
    for index, doc in enumerate(retrieved_documents):
        text = doc.text
        print(f"Retrived {index+1}: {text} \n")
        augmented_prompt += f"{text}\n"
    augmented_prompt += f"\nQuestion: {query}\nPlease provide a concise and accurate answer in ukrainian based on the context."
    return augmented_prompt


In [13]:
from IPython.display import Markdown, display

# Send the augmented prompt to a language model for a synthesized response
def get_RAG_response(query):    
    response = llm.complete(get_augmented_prompt(query))
    return response


In [ ]:
query = "створення системи цивільновійськового співробітництва"
response = get_RAG_response(query)
display(Markdown(f"## <b>Response:</b> \n #### {response}"))


Retrived 1: Анотація . На основі історично -ретроспективного аналізу, 
особистого досвіду участі у Антитерористичній операції на сході України 
у період 2014 -2015 р., автор визначає місце та важливу роль цивільно -
військових відносин/взаємодії у ході військового конфлікту (операції)  та 
подоланні наслідків ведення бойових дій, а також пост -конфліктний 
період, про необхідність існування та ефективного функціонування 
універсальної моделі цивільно -військових відносин у ЗСУ шляхом 
вдосконалення відносин між владою, армією і громадськістю.   
Ключові слова: цивільно -військове співробітництво (ЦВС), цивільно -
військові відносини, антитерористична операція (АТО) . 
Вступ  
Постановка проблеми. Цивільно -військові відносини, як вид 
соціальних взаємин, об’єктивно існують у будь -якому суспільстві і 
потре бують регулювання й упорядкування з боку держави для збереження 
сталості їх розвитку, забезпечення доцільності і конструктивізму, 
враховуючи особливі і спільні традиції цивільних 

## <b>Response:</b> 
 #### Створення системи цивільно-військового співробітництва (ЦВС) є важливим кроком для забезпечення ефективного функціонування відносин між владою, армією і громадськістю під час військових операцій та після них.

Проте такий механізм повинен включати в себе спеціальні програми розвитку співпраці між цивільними організаціями та військами (силами), а також створення ефективних структур управління для організації взаємодії між різними соціальними партнерами під час війни і після неї.